## Functions

This notebook has two sets of functions:
1. Functions formatting the graph traversal result as json. This is adapted from the great work done for the  activity browser. <br> https://github.com/LCA-ActivityBrowser/activity-browser
2. Functions setting the sankey diagram (dash) for results visualization

#### 1. Graph traversal

In [1]:
###################################################################################
########     Functions adapted from the Activity Browser open code  ###############

def get_json_data(data) -> str:
    """Transform bw.Graphtraversal() output for further use in Dash"""
    lca = data["lca"]
    lca_score = lca.score
    lcia_unit = bw.Method(lca.method).metadata["unit"]
    demand = list(lca.demand.items())[0]
    reverse_activity_dict = {v: k for k, v in lca.activity_dict.items()}

    build_json_node = compose_node_builder(lca_score, lcia_unit, demand[0])
    build_json_edge = compose_edge_builder(reverse_activity_dict, lca_score, lcia_unit)

    valid_nodes = (
        (bw.get_activity(reverse_activity_dict[idx]), v)
        for idx, v in data["nodes"].items() if idx != -1
    )
    valid_edges = (
        edge for edge in data["edges"]
        if all(i != -1 for i in (edge["from"], edge["to"]))
    )

    json_data = {
        "nodes": [build_json_node(act, v) for act, v in valid_nodes],
        "edges": [build_json_edge(edge) for edge in valid_edges],
    }

    return json_data
    
#@staticmethod
def compose_node_builder(lca_score: float, lcia_unit: str, demand: tuple):
    def build_json_node(act, values: dict) -> dict:
        return {
            "db": act.key[0],
            "id": act.key[1],
            "product": act.get("reference product") or act.get("name"),
            "name": act.get("name"),
            "location": act.get("location"),
            "amount": values.get("amount"),
            "LCIA_unit": lcia_unit,
            "ind": values.get("ind"),
            "ind_norm": values.get("ind") / lca_score,
            "cum": values.get("cum"),
            "cum_norm": values.get("cum") / lca_score,
            "class": "demand" if act == demand else identify_activity_type(act),
        }
    return build_json_node

#@staticmethod
def compose_edge_builder(reverse_dict: dict, lca_score: float, lcia_unit: str):
    def build_json_edge(edge: dict) -> dict:
        p = bw.get_activity(reverse_dict[edge["from"]])
        from_key = reverse_dict[edge["from"]]
        to_key = reverse_dict[edge["to"]]
        return {
            "source_id": from_key[1],
            "target_id": to_key[1],
            "amount": edge["amount"],
            "product": p.get("reference product") or p.get("name"),
            "impact": edge["impact"],
            "ind_norm": edge["impact"] / lca_score,
            "unit": lcia_unit,
            "tooltip": '{} ({:.2g} {})'
                        '{:.3g} {} ({:.2g}%) '.format(
                lcia_unit, edge["amount"], p.get("unit"),
                edge["impact"], lcia_unit, edge["impact"] / lca_score * 100,
            )
        }
    return build_json_edge

def identify_activity_type(activity):
    """Return the activity type based on its naming."""
    name = activity["name"]
    if "treatment of" in name:
        return "treatment"
    elif "market for" in name:
        # if not "to generic" in name:  # these are not markets, but also transferring activities
        return "market"
    elif "market group" in name:
        # if not "to generic" in name:
        return "marketgroup"
    else:
        return "production"

#### Dash-sankey function

In [ ]:
def set_activities(activities):
    "Create the list of sets for the activities dropdown in the Sankey diagram"
    name = []
    for act in activities:
        name.append(act['name'])
    options = [{'label': n,'value':n} for n in name]
        
    return options

In [ ]:
def set_methods(methods):
    "Create the list of sets for the methods dropdown in the Sankey diagram"
    name = []
    for m in methods:
        name.append(m)
    options = [{'label': n[1],'value':str(n)} for n in name]
        
    return options

In [7]:
def sankey(methods,activities):
    "Build Sankey diagram"
    
    app = JupyterDash(__name__) #working with jupyter
    #app = dash.Dash() #working with IDE
    
    ## Build dropdown menus
    app.layout = html.Div([
        ## Activities dropdown
        html.Div([
            html.Label(['Dropdown title (type your search or select from the list)']),
            dcc.Dropdown(
                id = 'process_dropdown',
                options=set_activities(activities),
                optionHeight = 35,
                #default value for the activity option
                value = activities[0]['name'],
                multi=False,
                clearable=False,
                searchable=True,
                search_value='',
                style={'width':'50%'}
            ),
        ]),
        ## Methods dropdown
        html.Div([
            html.Label(['Method']),
            dcc.Dropdown(
                id = 'method_dropdown',
                # modify options list to modify the impact categories evaluated
                options= set_methods(methods),
                optionHeight = 35,
                # impact category by default
                value = str(methods[0]),
                multi=False,
                clearable=False,
                searchable=True,
                search_value='',
                style={'width':'50%'}
            ),
        ]),
        
        ## Build cutoff dropdown
        html.Div([
            html.Label(['Cutoff (%) (type your search or select from the list)']),
            dcc.Dropdown(
                id = 'cutoff',
                # it is possible to modify this list of sets to modify the cutoff options
                options=[
                    {'label':'1%','value': '0.01'},
                    {'label':'2%','value': '0.02'},
                    {'label':'3%','value': '0.03'},
                    {'label':'4%','value': '0.04'},
                    {'label':'5%','value': '0.05'},
                    {'label':'6%','value': '0.06'},
                    {'label':'7%','value': '0.07'},
                    {'label':'8%','value': '0.08'},
                    {'label':'9%','value': '0.09'},
                    {'label':'10%','value': '0.10'},
                    ],
                optionHeight = 35,
                # cutoff value by default
                value = '0.05',
                multi=False,
                clearable=False,
                searchable=True,
                search_value='5%',
                style={'width':'50%'}
            ),
        ]),
            
        html.Div([
            dcc.Graph(id='sankey')
        ])
    ])

    @app.callback(
        Output(component_id='sankey', component_property='figure'),
        [Input(component_id='process_dropdown', component_property='value'),
         Input(component_id='method_dropdown', component_property='value'),
         Input(component_id='cutoff', component_property='value')]
    )

    #%% Nested function - Perform LCA calculation, transverse and match edges and nodes colors
    ### TO-DO: This could be speed up - No need for LCA calculation each time we do a callback
    def update_graph(process_dropdown,method_dropdown,cutoff_dropdown):
        process = [act for act in eidb if process_dropdown in act['name'] ][0]
        fu = {process:1}
        m = [method for method in bw.methods if method_dropdown in str(method)][0]
        cutoff = float(cutoff_dropdown)
    
        data = bw.GraphTraversal().calculate(fu, m, cutoff=cutoff, max_calc=10000)
        process_dict = get_json_data(data)

        edges = pd.DataFrame.from_dict(process_dict['edges'])
        nodes = pd.DataFrame.from_dict(process_dict['nodes'])
    
        mapping_links = {k: v for v, k in enumerate(nodes.id.unique())}
        edges['source_index'] = edges.source_id.map(mapping_links)
        edges['target_index'] = edges.target_id.map(mapping_links)
    
        colors = []
        colors_map = dict()
        key=0
        for row in nodes.iterrows():
            r = np.random.randint(255)
            g = np.random.randint(255)
            b = np.random.randint(255)
            a = 0.4
            color_row = (f"rgba({r},{g},{b},{a})")
            colors.append(color_row)
        for color in colors:
            colors_map.update({key:color})
            key+=1
        nodes['color'] = colors
        edges['color'] = edges.source_index.map(colors_map)
    
        fig = go.Figure(go.Sankey(
            valueformat = ".0f",
            valuesuffix = "%",
            #arrangement = "snap",
            node = {
                "label": nodes['product'],
                "color": nodes['color'],
                #"x": ,
                #"y": [0.7, 0.5, 0.2, 0.4, 0.2, 0.3],
                'pad':10},  # 10 Pixels,
            link = {
                "source": edges['source_index'],
                "target": edges['target_index'],
                "color": edges['color'],
                "value": 100 * edges['ind_norm']}))
        fig.update_layout(title_text="Contribution analysis - Sankey diagram.", font_size=8, height = 800)
        
        return (fig)
        
    return app

In [ ]:
#         fig = go.Figure(data = [go.Sankey(
#             valueformat = ".0f",
#             valuesuffix = str(edges[0]['unit']),
#             #arrangement = "snap",
#             node = {
#                 "label": nodes['product'],
#                 "color": nodes['color'],
#                 #"x": ,
#                 #"y": [0.7, 0.5, 0.2, 0.4, 0.2, 0.3],
#                 'pad':10},  # 10 Pixels,
#             link = {
#                 "source": edges['source_index'],
#                 "target": edges['target_index'],
#                 "color": edges['color'],
#                 "value": edges['ind_norm']})])
#         fig.update_layout(title_text="Contribution analysis - Sankey diagram.", font_size=8, height = 800)

In [ ]:
#         fig = go.Figure(go.Sankey(

#             #arrangement = "snap",
#             node = {
#                 "label": nodes['product'],
#                 "color": nodes['color'],
#                 #"x": ,
#                 #"y": [0.7, 0.5, 0.2, 0.4, 0.2, 0.3],
#                 'pad':10},  # 10 Pixels,
#             link = {
#                 "source": edges['source_index'],
#                 "target": edges['target_index'],
#                 "color": edges['color'],
#                 "value": edges['ind_norm']}))
#         fig.update_layout(title_text="Contribution analysis - Sankey diagram.", font_size=8, height = 800)